# Evaluating  Landsat-8  and  Sentinel-2  Nadir  BRDF  Adjusted  Reflectance (NBAR) on South of Brazil through a Reproducible and Replicable Workflow

#### **Authors**: Rennan de Freitas Bezerra Marujo (https://orcid.org/0000-0002-0082-9498), Felipe M. Carlos (0000-0002-3334-4315), Raphael W. Costa, Carlos A. F. Noronha, José Guilherme Fronza (https://orcid.org/0000-0002-0830-8101), Jeferson de Souza Arcanjo (), Anderson Soares (https://orcid.org/0000-0001-6513-2192), Gilberto R. Queiroz (https://orcid.org/0000-0001-7534-0219) and Karine R. Ferreira (https://orcid.org/0000-0003-2656-5504)

This notebook runs a workflow of the article: "Evaluating  Landsat-8  and  Sentinel-2  Nadir  BRDF  Adjusted  Reflectance (NBAR) on South of Brazil through a Reproducible andReplicable Workflow".

The main idea of the article consists in comparing Lansat-8/OLI and Sentinel-2/MSI Surface images (SR) and Nadir BRDF  Adjusted  Reflectance (NBAR).

## 1. Resource configuration

The first step of the Workflow consists in configuring the input directories, images to be processed, auxiliary data and output directories.

In [1]:
import os

### 1.1 Configure data directory (with environment variable)

This directory is the root of the Workflow and will be used as basepath for the remaining directories.

In [4]:
data_directory = os.environ.get("DATA_DIRECTORY")

data_directory

### 1.2 Defining input files and directories for Sentinel-2/MSI steps

`sentinel2_input_dir` points to the path containing the Sentinel-2 L1C (Level-1 C, Top of Atmosphere) folders (.SAFE), which were originally obtained from Copernicus scihub (https://scihub.copernicus.eu/dhus/#/home).

`sentinel2_sceneid_list` points to a `.txt` file containing all Sentinel-2 sceneids that will be processed (separated by line breaks).

In [5]:
sentinel2_input_dir = os.path.join(data_directory, "raw_data/Sentinel2Data")

sentinel2_sceneid_list = os.path.join(data_directory, "raw_data/ReferenceFiles_L8-S2/s2-sceneids.txt")

### 1.3 Defining input files and directories for Landsat-8/OLI steps

`landsat8_input_dir` variable points to the path containing the Landsat-8 Collection-2 Level-2 (Surface Reflectance) folders, which were originally obtained from USGS Earth Explorer (http://earthexplorer.usgs.gov/).

`landsat8_sceneid_list` points to a `.txt` file containing all Landsat-8 sceneids that will be processed (separated by line breaks).

In [6]:
landsat8_input_dir = os.path.join(data_directory, "raw_data/Landsat8Data")

landsat8_sceneid_list = os.path.join(data_directory, "raw_data/ReferenceFiles_L8-S2/l8-sceneids.txt")

### 1.4 Defining LaSRC auxiliary data

The Sentinel-2 L1C will be processed to Surface Reflectance throgh LaSRC and Sen2cor Atmospheric Correction. LaSRC requires a few auxiliary files. Auxiliary files for the correction can be obtained at https://edclpdsftp.cr.usgs.gov/downloads/auxiliaries/lasrc_auxiliary/L8/. Inside the auxiliary data, there is the LADS, which are daily files required to perform the atmosphere correction. In order to correct the Sentinel-2 images using LaSRC, you should have the auxiliary files and for each Sentinel-2 sceneid that is going to be processed, you also should have the LADS correspondent to the sceneid sensing date.

`lads_auxiliary_data` points to the folder containing the LADS files, which are part of the LaSRC auxiliary files.

In [7]:
lads_auxiliary_data = os.path.join(data_directory, "raw_data/LADS_AuxiliaryData/")

### 1.5 Defining output directory

All the results of this Workflow, which includes processed images and result dataframes, will be stored in the `derived_data_dir` in a organized separation.

`derived_data_dir` is the root folder of the processed data.

In [8]:
derived_data_dir = os.path.join(data_directory, "derived_data")

### 1.6 Load scene identifiers (Landsat-8/OLI and Sentinel-2/MSI)

Once all the directories are set, we can load the sceneids list containing all images that will be processed and analysed

`landsat8_sceneids` contains all the Landsat-8 sceneids that will be used

`sentinel2_sceneids` contains all the Sentinel-2 sceneids that will be used

In [9]:
from research_processing import toolbox

ModuleNotFoundError: No module named 'research_processing'

In [ ]:
landsat8_sceneids = toolbox.standardize_filename(open(landsat8_sceneid_list).readlines())

sentinel2_sceneids = toolbox.standardize_filename(open(sentinel2_sceneid_list).readlines())

## 1.7 Obtaining output scenes

`l8_outputs` points to the folder in which the Landsat-8 results will be stored.

`s2_outputs` points to the folder in which Sentinel-2 results will be stored.

`analysis_outputs` points to the folder in which the comparison analysis will be stored.

In [ ]:
l8_outputs = os.path.join(derived_data_dir, "l8")

s2_outputs = os.path.join(derived_data_dir, "s2")

analysis_outputs = os.path.join(derived_data_dir, "analysis")

## 2. Atmosphere Correction

For this study, Landsat-8 images were already acquired as Surface Reflectance products.

However, Sentinel-2 images were obtained as L1C (Top of Atmosphere). To compare Sentinel and Landsat-8 first we must process Sentinel-2 to Surface Reflectance. For that we opted for using 2 methods:

- `Sen2cor`: is provided by the European Space Agency (ESA) and consist in a processor for Sentinel-2 Level 2A product generation

- `LaSRC`: The Landsat 8 Collection 1 Land Surface Reflectance Code (LaSRC) software uses an implementation of the Second Simulation of a Satellite Signal in the Solar Spectrum (6S) atmospheric correction model, to correct the atmospheric effects. It was originally projected to correct Landsat-8 Collection-1 images, but was adapted to also runs Sentinel-2 images.

On this section we perform the atmosphere correction on Sen2cor images.

### 2.1. Sentinel-2/MSI (Sen2Cor)

To process the Sentinel-2 L1C images using `Sen2cor`, first lets import the module. 

In [ ]:
from research_processing.surface_reflectance import sen2cor

#### 2.1.1 Prepare output directory

`sen2cor_dir` points to the path were the Sentinel-2 L2A (Level-2 A, Surface Reflectance) folders (.SAFE) will be outputed. 

In [ ]:
sen2cor_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_sr")

#### 2.1.2 Apply sen2cor

Now we can perform the atmosphere correction using Sen2cor.

*note*: it can take a while to process (more than 20 minutes per image).

In [ ]:
sen2cor(sentinel2_input_dir, sen2cor_dir, sentinel2_sceneids)

#### 2.1.3 Saving Sen2Cor scene list

L2A Folder Names are differente from L1C Folder Names, due to the processing date.
Here we obtain the L2A Folder names.

In [ ]:
sen2cor_sceneids = toolbox.filename(os.listdir(sen2cor_dir))

### 2.2. Sentinel-2/MSI (LaSRC)

As performed using `Sen2cor`, here we atmospherically correct the Sentinel-2 L1C to produce Surface Reflectance products, but correcting through LaSRC.

In [ ]:
from research_processing.surface_reflectance import lasrc

#### 2.2.1 Prepare output directory

`lasrc_dir` points to the path were the Sentinel-2 LaSRC Surface Reflectance folders will be outputed. 

In [ ]:
lasrc_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_sr")

#### 2.2.2 Apply LaSRC

Now we can perform the atmosphere correction using LaSRC.

*note*: it can take a while to process (more than 20 minutes per image).

In [ ]:
lasrc(sentinel2_input_dir, lasrc_dir, sentinel2_sceneids, lads_auxiliary_data)

#### 2.2.3 Saving LaSRC scene list

Here we obtain the processed images

In [ ]:
lasrc_sceneids = toolbox.filename(os.listdir(lasrc_dir))

## 3. NBAR

Remote sensing systems observations are influenced by the Sun-Sensor Geometry Effects, which are described by the Bidirectional Reflectance Distribution Function (BRDF). The surface reflectance measured by an orbital sensor is influenced by changes on the Sun-Sensor geometry due to differences in azimuth and zenith angles. Scattering models can be used to estimate Bidirectional Reflectance Factors (BRFs) as if they were observed in a specific view and sun zenith angles. Nadir BRDF-Adjusted Reflectance (NBAR) products are surface reflectance estimated through a scattering model at nadir (0&deg; view zenith) and a specified solar zenith angle.

On this section we perform the NBAR correction on Sen2cor and Landsat images.

### 3.1. Sentinel-2/MSI (Sen2Cor) NBAR

In [ ]:
from research_processing.nbar import s2_sen2cor_nbar

#### 3.1.1 Prepare output directory

`s2_sen2cor_nbar_dir` points to the path were the Sentinel-2 NBAR folders will be outputed.

In [ ]:
s2_sen2cor_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_nbar")

#### 3.1.2 Generate NBAR for Sentinel-2 (Sen2cor) products

Now we can generate the NBAR on the Surface Reflectance products (generated through Sen2cor).

*note:* when generating Sentinel-2 NBAR, an internal python package generates the Angle bands, due to that here we do not need to explicity call the angle generation.

In [ ]:
s2_sen2cor_nbar(sen2cor_dir, s2_sen2cor_nbar_dir, sen2cor_sceneids)

#### 3.1.3 Saving Sen2Cor NBAR scene list

In [ ]:
s2_sen2cor_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))

### 3.2. Sentinel-2/MSI (LaSRC) NBAR

In [ ]:
from research_processing.nbar import s2_lasrc_nbar

#### 3.2.1 Prepare output directory

`s2_lasrc_nbar_dir` points to the path were the Sentinel-2 (LaSRC) NBAR folders will be outputed.

In [ ]:
s2_lasrc_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_nbar")

#### 3.2.2 Generate NBAR for Sentinel-2 (LaSRC) products

Now we can generate the NBAR on the Surface Reflectance products (generated through LaSRC).

*note:* when generating Sentinel-2 NBAR, an internal python package generates the Angle bands, due to that here we do not need to explicity call the angle generation.

In [ ]:
s2_lasrc_nbar(lasrc_dir, s2_lasrc_nbar_dir, lasrc_sceneids)

#### 3.2.3 Saving LaSRC NBAR scene list

In [ ]:
s2_lasrc_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))

### 3.3. Landsat-8/OLI NBAR

In [ ]:
from research_processing.nbar import lc8_nbar, lc8_generate_angles

#### 3.3.1 Prepare output directory

`lc8_nbar_dir` points to the path were the Landsat-8 NBAR folders will be outputed.

In [ ]:
lc8_nbar_dir = toolbox.prepare_output_directory(l8_outputs, "lc8_nbar")

#### 3.3.2 Generate Landsat-8 Angles for NBAR calculation

Here we use the USGS Angle Creation Tool to obtain the angles for the Ladsat-8 images.

In [ ]:
scene_angles_lc8 = lc8_generate_angles(landsat8_input_dir, landsat8_sceneids)

#### 3.3.3 Generate NBAR for Landsat-8/OLI Surface Reflectance products

Now we can generate the NBAR on the Surface Reflectance products.

In [ ]:
lc8_nbar(landsat8_input_dir, lc8_nbar_dir, scene_angles_lc8)

#### 3.3.4 Saving Landsat-8/OLI NBAR scene list

In [ ]:
lc8_nbar_scene_ids = toolbox.filename(os.listdir(lc8_nbar_dir))

## 4. Analysis

On this section we will perform the analyses, which are performed considering single sensors:

- SR: Landsat-8
- NBAR: Landsat-8
- SR: Sentinel-2 (Sen2Cor)
- NBAR: Sentinel-2 (Sen2Cor)
- SR: Sentinel-2 (LaSRC)
- NBAR: Sentinel-2 (LaSRC)

and also considering both Landsat-8 and Sentinel-2 together:
- SR: Landsat-8 x Sentinel-2 (Sen2cor)
- SR: Landsat-8 x Sentinel-2 (LaSRC)
- NBAR: Landsat-8 x Sentinel-2 (Sen2Cor)
- NBAR: Landsat-8 x Sentinel-2 (LaSRC)

To do that, first we create pairs of images. This pairing uses images close in time, so for Sentinel-2 we assume that images with a 5 day difference can be compared, while Landsat-8 images within a 10 day difference is also aceptable and when comparing both sensor we also use a maximum difference of 5 days.
The analysis is performed by comparing the pixel values within images overlap and disconsidering pixels masked as cloud, cloud shadow or snow in any of the images being compared. The comparison is performed band-wise.

In [ ]:
from research_processing.validation import validation_funcs
from research_processing.validation import validation_routines

### 4.1. Analysis SR: Landsat-8

Here the Surface Reflectance of Landsat-8 is analysed.

#### 4.1.1 Search pairs

Images with a maximum sensing date difference of 10 days are selected

In [ ]:
validation_sr_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list)

#### 4.1.2 Prepare output directory

In [ ]:
validation_sr_l8_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8")

#### 4.1.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

#### 4.1.4 Validate the results

In [ ]:
validation_sr_l8_result = validation_routines.validation_sr_l8(
    input_dir  = landsat8_input_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_sr_l8_dir,
    pairs      = validation_sr_l8_pairs, 
    bands      = validation_sr_l8_bands
)

### 4.2. Analysis NBAR: Landsat-8

Here the NBAR of Landsat-8 is analysed.

#### 4.2.1 Search pairs

Images with a maximum sensing date difference of 10 days are selected

In [ ]:
validation_nbar_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list)

#### 4.2.2 Prepare output directory

In [ ]:
validation_nbar_l8_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8")

#### 4.2.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

#### 4.2.4 Validate the results

In [ ]:
validation_nbar_l8_results = validation_routines.validation_nbar_l8(
    input_dir  = lc8_nbar_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_nbar_l8_dir,
    pairs      = validation_nbar_l8_pairs,
    bands      = validation_nbar_l8_bands
)

### 4.3. Analysis SR: Sentinel-2 (Sen2Cor)

Here the SR of Sentinel-2 (Sen2cor) is analysed.

#### 4.3.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.3.2 Prepare output directory

In [ ]:
validation_sr_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_s2_sen2cor")

#### 4.3.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_sr_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_sr_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

#### 4.3.4 Validate the results

In [ ]:
validation_routines.validation_sr_s2_sen2cor(
    input_dir  = sen2cor_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_sen2cor_dir,
    pairs      = validation_sr_s2_sen2cor_pairs,
    bands10m   = validation_sr_s2_sen2cor_bands10m,
    bands20m   = validation_sr_s2_sen2cor_bands20m
)

### 4.4. Analysis NBAR: Sentinel-2 (Sen2Cor)

Here the NBAR of Sentinel-2 (Sen2cor) is analysed.

#### 4.4.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.4.2 Prepare output directory

In [ ]:
validation_nbar_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_s2_sen2cor")

#### 4.4.3 Defining bands

Name of the spectral bands that will be used in the validation. 

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_nbar_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_nbar_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

#### 4.4.4 Validate the results

In [ ]:
validation_routines.validation_nbar_s2_sen2cor(
    input_dir  = s2_sen2cor_nbar_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_nbar_s2_sen2cor_dir,
    pairs      = validation_nbar_s2_sen2cor_pairs,
    bands10m   = validation_nbar_s2_sen2cor_bands10m,
    bands20m   = validation_nbar_s2_sen2cor_bands20m
)

### 4.5. Analysis SR: Sentinel-2 (LaSRC)

Here the SR of Sentinel-2 (LaSRC) is analysed.

#### 4.5.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.5.2 Prepare output directory

In [ ]:
validation_sr_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_s2_lasrc")

#### 4.5.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_sr_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.5.4 Validate the results

In [ ]:
validation_routines.validation_sr_s2_lasrc(
    input_dir  = lasrc_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_lasrc_dir,
    pairs      = validation_sr_s2_lasrc_pairs,
    bands      = validation_sr_s2_lasrc_bands
)

### 4.6. Analysis NBAR: Sentinel-2 (LaSRC)

Here the NBAR of Sentinel-2 (LaSRC) is analysed.

#### 4.6.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.6.2 Prepare output directory

In [ ]:
validation_nbar_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_s2_lasrc")

#### 4.6.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_nbar_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.6.4 Validate the results

In [ ]:
validation_routines.validation_nbar_s2_lasrc(
    input_dir  = s2_lasrc_nbar_dir, 
    cloud_dir  = sen2cor_dir, 
    output_dir = validation_nbar_s2_lasrc_dir, 
    pairs      = validation_nbar_s2_lasrc_pairs,
    bands      = validation_nbar_s2_lasrc_bands
)

### 4.7. Analysis SR: Landsat-8 x Sentinel-2 (Sen2Cor)

Here the SR of Landsat-8 and Sentinel-2 (Sen2cor) are analysed.

#### 4.7.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.7.2 Prepare output directory

In [ ]:
validation_sr_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8_s2_sen2cor")

#### 4.7.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

#### 4.7.4 Validate the results

In [ ]:
validation_routines.validation_sr_l8_s2_sen2cor(
    input_dir_l8 = landsat8_input_dir, 
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = sen2cor_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_sen2cor_dir,
    pairs        = validation_sr_l8_s2_sen2cor_pairs,
    bands_l8     = validation_sr_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_sr_l8_s2_sen2cor_bands_s2
)

### 4.8. Analysis SR: Landsat-8 x Sentinel-2 (LaSRC)¶

Here the SR of Landsat-8 and Sentinel-2 (LaSRC) are analysed.

#### 4.8.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.8.2 Prepare output directory

In [ ]:
validation_sr_l8_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8_s2_lasrc")

#### 4.8.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_sr_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.8.4 Validate the results

In [ ]:
validation_routines.validation_sr_l8_s2_lasrc(
    input_dir_l8 = landsat8_input_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = lasrc_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_lasrc_dir,
    pairs        = validation_sr_l8_s2_lasrc_pairs,
    bands_l8     = validation_sr_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_sr_l8_s2_lasrc_bands_s2
)

### 4.9. Analysis NBAR: Landsat-8 x Sentinel-2 (Sen2Cor)

Here the NBAR of Landsat-8 and Sentinel-2 (Sen2cor) are analysed.

#### 4.9.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.9.2 Prepare output directory

In [ ]:
validation_nbar_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8_s2_sen2cor")

#### 4.9.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

#### 4.9.4 Validate the results

In [ ]:
validation_routines.validation_nbar_l8_s2_sen2cor(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_sen2cor_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_sen2cor_dir,
    pairs        = validation_nbar_l8_s2_sen2cor_pairs,
    bands_l8     = validation_nbar_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_nbar_l8_s2_sen2cor_bands_s2
)

### 4.10. Analysis NBAR Landsat-8 x Sentinel-2 (LaSRC)¶

Here the NBAR of Landsat-8 and Sentinel-2 (LaSRC) are analysed.

#### 4.10.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.10.2 Prepare output directory

In [ ]:
validation_nbar_l8_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8_s2_lasrc")

#### 4.10.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_nbar_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.10.4 Validate the results

In [ ]:
validation_routines.validation_nbar_l8_s2_lasrc(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_lasrc_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_lasrc_dir, 
    pairs        = validation_nbar_l8_s2_lasrc_pairs,
    bands_l8     = validation_nbar_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_nbar_l8_s2_lasrc_bands_s2
)

## 5. Saving results table

In this section, the analysis results are inserted into a tidy structure and saved as a `.csv`. 

In [ ]:
from research_processing.tidy import map_validation_folder_as_tidydata

#### 5.1 Generating the tidy results

In [ ]:
tidy_output = map_validation_folder_as_tidydata(validation_outputs)
tidy_output

5.2 Saving the results

In [ ]:
tidy_output.to_csv(os.path.join(validation_outputs, "results.csv"))